# Transfer Learning with Tensorflow part 2: Fine-Tuning

In [1]:
# Check if we're using GPU
!nvidia-smi


Fri Jan 12 15:15:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Creating helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-01-12 15:17:55--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-12 15:17:55 (86.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import *

## Load the data

In [4]:
# get 10% of training data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2024-01-12 15:24:42--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   217MB/s    in 0.7s    

2024-01-12 15:24:43 (217 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
unzip_data('10_food_classes_10_percent.zip')

In [6]:
# Check out how many images and subdirectories are in our dataset
walk_through_dir('10_food_classes_10_percent/')

There are 2 directories and 0 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/tra

In [7]:
# Create training and test directory paths
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [10]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=BATCH_SIZE,
                                                                            shuffle=True)

test_data= tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                               image_size=IMG_SIZE,
                                                               label_mode='categorical',
                                                               batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [11]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [14]:
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[ 12.642858    4.642857    1.6428572]
   [ 12.97449     4.9744897   1.9744899]
   [ 14.857143    6.857143    3.857143 ]
   ...
   [254.40312   247.8422    241.41367  ]
   [251.81125   254.66837   245.52548  ]
   [245.68332   247.96896   230.61179  ]]

  [[ 14.          7.          1.       ]
   [ 14.          7.          1.       ]
   [ 14.285714    7.285714    1.2857141]
   ...
   [255.        253.6379    247.79614  ]
   [251.14798   255.        248.58669  ]
   [242.16798   242.45363   226.93309  ]]

  [[ 17.428572    8.428572    3.4285717]
   [ 15.5         6.5         1.5      ]
   [ 15.214286    6.214286    1.2142859]
   ...
   [251.59688   254.23477   248.50017  ]
   [249.94391   255.        249.51526  ]
   [246.66301   244.07622   231.2343   ]]

  ...

  [[167.4439     48.872425    7.219402 ]
   [169.28067    53.209236    6.4235206]
   [174.96925    59.39783     7.7549734]
   ...
   [250.16835   141.16835    58.168354 ]
   [247.87242   138.87242    53.872425 ]
   [2

In [23]:
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))
base_model.trainable = False

inputs=tf.keras.Input(shape=IMG_SIZE+(3,))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, output)

In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [25]:
model.fit(train_data_10_percent,
          epochs=5,
          validation_data=test_data,
          steps_per_epoch=len(train_data_10_percent),
          validation_steps=len(test_data))

Epoch 1/5
24/24 [==============================] - 22s 483ms/step - loss: 1.8502 - accuracy: 0.4413 - val_loss: 1.2926 - val_accuracy: 0.7088
Epoch 2/5
24/24 [==============================] - 7s 285ms/step - loss: 1.0595 - accuracy: 0.7640 - val_loss: 0.8700 - val_accuracy: 0.8088
Epoch 3/5
24/24 [==============================] - 8s 311ms/step - loss: 0.7556 - accuracy: 0.8333 - val_loss: 0.7026 - val_accuracy: 0.8348
Epoch 4/5
24/24 [==============================] - 8s 344ms/step - loss: 0.6101 - accuracy: 0.8533 - val_loss: 0.6140 - val_accuracy: 0.8508
Epoch 5/5
24/24 [==============================] - 13s 528ms/step - loss: 0.5180 - accuracy: 0.8840 - val_loss: 0.5607 - val_accuracy: 0.8572
